# 분류 및 연속모델 복합 구조 cVAE를 이용한 촉매 조성 최적화
### (KL Annealing 제거 및 3단계 계층적 Gated 구조 반영)

In [ ]:
import os, sys, torch, json
import numpy as np
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score

# 커스텀 모듈 경로 추가
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
from multilayer_model.m3_multi_bce import M3_Multi_BCE
from multilayer_model.m3_multi_mse import M3_Multi_MSE
from multilayer_loss.l_multi3_final_logic import l_multi3_final_loss
from vae_earlystopping import EarlyStopping

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 800
print(f"Active Device: {device}")

In [ ]:
# 1. 데이터 로드 (4개 파일 통합)
x1_raw = np.load('../data/metal.npy')
x2_raw = np.load('../data/support_norm.npy')
x3_raw = np.load('../data/pre_fin_norm.npy')
c_raw = np.load('../data/re_fin.npy')

# 데이터 분할 (8:1:1)
indices = np.arange(len(x1_raw))
tr_idx, te_idx = train_test_split(indices, test_size=0.1, random_state=42)
tr_idx, va_idx = train_test_split(tr_idx, test_size=0.1, random_state=42)

sc1, sc2, sc3, scc = MinMaxScaler(), MinMaxScaler(), MinMaxScaler(), MinMaxScaler()
def prep(d, i, s, fit=False): return s.fit_transform(d[i]) if fit else s.transform(d[i])

x1_tr, x1_va, x1_te = prep(x1_raw, tr_idx, sc1, True), prep(x1_raw, va_idx, sc1), prep(x1_raw, te_idx, sc1)
x2_tr, x2_va, x2_te = prep(x2_raw, tr_idx, sc2, True), prep(x2_raw, va_idx, sc2), prep(x2_raw, te_idx, sc2)
x3_tr, x3_va, x3_te = prep(x3_raw, tr_idx, sc3, True), prep(x3_raw, va_idx, sc3), prep(x3_raw, te_idx, sc3)
c_tr, c_va, c_te = prep(c_raw, tr_idx, scc, True), prep(c_raw, va_idx, scc), prep(c_raw, te_idx, scc)

def to_t(a): return torch.tensor(a, dtype=torch.float32)
train_loader = DataLoader(TensorDataset(to_t(x1_tr), to_t(x2_tr), to_t(x3_tr), to_t(c_tr)), batch_size=64, shuffle=True)
val_loader = DataLoader(TensorDataset(to_t(x1_va), to_t(x2_va), to_t(x3_va), to_t(c_va)), batch_size=64)
test_loader = DataLoader(TensorDataset(to_t(x1_te), to_t(x2_te), to_t(x3_te), to_t(c_te)), batch_size=64)

x_dims = [x1_tr.shape[1], x2_tr.shape[1], x3_tr.shape[1]]
c_dim = c_tr.shape[1]

In [ ]:
def train_model(model, mode='mse'):
    optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
    es = EarlyStopping(patience=40, min_delta=1e-9)
    
    print(f"\n--- Training {mode.upper()} Specialist ---")
    for epoch in range(1, EPOCHS + 1):
        model.train()
        train_loss = 0
        for b1, b2, b3, bc in train_loader:
            b1, b2, b3, bc = b1.to(device), b2.to(device), b3.to(device), bc.to(device)
            optimizer.zero_grad()
            preds, mus, lvs = model(b1, b2, b3, bc)
            # l_multi3_final_loss 내부에서 annealing 없이 gamma Balancing만 수행
            loss = l_multi3_final_loss(preds, [b1, b2, b3], mus, lvs, mode=mode)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for v1, v2, v3, vc in val_loader:
                v1, v2, v3, vc = v1.to(device), v2.to(device), v3.to(device), vc.to(device)
                vp, vm, vv = model(v1, v2, v3, vc)
                val_loss += l_multi3_final_loss(vp, [v1, v2, v3], vm, vv, mode=mode).item()
        
        avg_val_loss = val_loss / len(val_loader)
        if epoch % 100 == 0:
            print(f"Epoch {epoch} | Val Loss: {avg_val_loss:.6f}")
        
        if es(avg_val_loss, model):
            print(f"Early stopping at epoch {epoch}")
            break
            
    es.load_best_model(model)
    return model

# BCE 모델 학습
model_bce = train_model(M3_Multi_BCE(x_dims, c_dim).to(device), mode='bce')

# MSE 모델 학습
model_mse = train_model(M3_Multi_MSE(x_dims, c_dim).to(device), mode='mse')

In [ ]:
print("\n--- Performing Final Gated Evaluation (Phase 3: Pretreatment) ---")
model_bce.eval()
model_mse.eval()

all_preds, all_trues = [], []

with torch.no_grad():
    for b1, b2, b3, bc in test_loader:
        bc = bc.to(device)
        
        # 1. BCE에서 생성 (Prior Sampling)
        b_gen = model_bce.generate(bc, device)
        prob3 = torch.sigmoid(b_gen[2]) # Pretreatment 존재 확률
        
        # 2. MSE에서 생성 (Prior Sampling)
        m_gen = model_mse.generate(bc, device)
        val3 = m_gen[2] # Pretreatment 수치 함량
        
        # 3. Gated Combine: Probability * Value
        combined = (prob3 * val3).cpu().numpy()
        
        # 4. Inverse Scaling
        preds_inv = sc3.inverse_transform(combined)
        trues_inv = sc3.inverse_transform(b3.cpu().numpy())

        all_preds.append(preds_inv)
        all_trues.append(trues_inv)

y_true = np.concatenate(all_trues).flatten()
y_pred = np.concatenate(all_preds).flatten()

final_r2 = r2_score(y_true, y_pred)
print(f"\n🏆 Final R2 Score (Pretreatment Stage): {final_r2:.4f}")

# 시각화
import matplotlib.pyplot as plt
plt.figure(figsize=(8, 6))
plt.scatter(y_true, y_pred, alpha=0.5, color='teal')
plt.plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'r--', label='Ideal')
plt.title(f"Pretreatment Generation Accuracy (R2: {final_r2:.4f})")
plt.xlabel("Actual Values"); plt.ylabel("Generated Values")
plt.legend(); plt.grid(True); plt.show()